In [1]:
# pip install ortools

In [3]:
import time
"""Simple Travelling Salesperson Problem (TSP) between cities."""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = [
        [
            0, 84, 72, 72, 56, 75, 9, 11, 14, 4, 8, 12, 10, 6
        ],
        [
            84, 0, 132, 157, 141, 148, 75, 99, 110, 79, 84, 89, 87, 85
        ],
        [
            72, 132, 0, 69, 79, 126, 66, 77, 74, 87, 123, 88, 80, 100
        ],
        [
            72, 157, 69, 0, 20, 70, 92, 76, 67, 71, 67, 64, 63, 67
        ],
        [
            56, 141, 79, 20, 0, 53, 60, 61, 47, 55, 51, 48, 51, 51
        ],
        [
            75, 148, 126, 70, 53, 0, 80, 60, 79, 66, 68, 67, 60, 62
        ],
        [
            9, 75, 66, 92, 60, 80, 0, 18, 18, 5, 10, 14, 13, 11
        ],
        [
            11, 99, 77, 76, 61, 60, 18, 0, 12, 12, 10, 14, 11,7
        ],
        [
            14, 110, 74, 67, 47, 79, 18, 12, 0, 14, 10, 14, 8, 9
        ],
        [
            4, 79, 87, 71, 55, 66, 5, 12, 14, 0, 5, 8, 8, 7
        ],
        [
            8, 84, 123, 67, 51, 68, 10, 10, 10, 5, 0, 5, 4, 3
        ],
        [
            12, 89, 88, 64, 48, 67, 14, 14, 14, 8, 5, 0, 5, 7
        ],
        [
            10, 87, 80, 63, 47, 60, 13, 11, 8, 8, 4, 5, 0, 4
        ],
        [
            6, 85, 100, 67, 51, 62, 11, 7, 9, 7, 3, 7, 4, 0
        ],
    ]  # yapf: disable
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data


def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print('Total distance: {} km'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}miles\n'.format(route_distance)


def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(manager, routing, solution)


if __name__ == '__main__':
    start_time = time.time()
    main()
    print("---------------")
    print("Total time taken :: %s seconds" % (time.time() - start_time))

Total distance: 457 km
Route for vehicle 0:
 0 -> 13 -> 10 -> 11 -> 12 -> 8 -> 7 -> 5 -> 4 -> 3 -> 2 -> 1 -> 6 -> 9 -> 0

---------------
Total time taken :: 0.0062160491943359375 seconds
